In [1]:
import numpy as np
import pickle
import cartopy.io.shapereader as shpreader
from cartopy.feature import ShapelyFeature
from shapely.prepared import prep
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pandas as pd

In [2]:
# some useful arrays to translate between naming conventions

election_year_list = np.array([1992, 1994, 1996, 1998, 2000, 2002, 2004, 2006, 2008, 2010, 2012, 
                                   2014, 2016, 2018])
congress_ID_list = np.array([103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116])

state_names = np.array(['ALABAMA', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA', 
               'COLORADO', 'CONNECTICUT', 'DELAWARE', 'FLORIDA', 'GEORGIA', 
               'HAWAII', 'IDAHO', 'ILLINOIS', 'INDIANA', 'IOWA', 'KANSAS', 
               'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND', 'MASSACHUSETTS', 
               'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI', 'MISSOURI', 'MONTANA', 
               'NEBRASKA', 'NEVADA', 'NEW HAMPSHIRE', 'NEW JERSEY', 'NEW MEXICO', 
               'NEW YORK', 'NORTH CAROLINA', 'NORTH DAKOTA', 'OHIO', 'OKLAHOMA', 
               'OREGON', 'PENNSYLVANIA', 'RHODE ISLAND', 'SOUTH CAROLINA', 
               'SOUTH DAKOTA', 'TENNESSEE', 'TEXAS', 'UTAH', 'VERMONT', 
               'VIRGINIA', 'WASHINGTON', 'WEST VIRGINIA', 'WISCONSIN', 'WYOMING'])

state_abbrs = np.array(['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL',
              'IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT',
              'NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI',
              'SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY'])

state_fips = np.array([1, 2, 4, 5, 6, 8, 9, 10, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 
                       24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 
                       41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56])

In [3]:
# The proj.4 string:
# +proj=longlat +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +no_defs

In [4]:
def read_shapefiles(election_years,verbose=True):
    """
    Reads in shapefiles from UCLA database (pre-2016) US Census TigerLine files (2016 on).
    data sources: http://cdmaps.polisci.ucla.edu/
                  https://www.census.gov/geo/maps-data/data/cbf/cbf_cds.html
    Note that file names must be in the same folder as the code with the following format:
        -pre-2016: districtShapes-NNN/districtsNNN.shp
        -2016 on: must be named t1_YYYY_us_cdNNN/tl_YYYY_us_cdNNN.shp
    ...where NNN is congress ID and YYYY is election year
    
    input:
        election_years -- (list) Elections years of the shape files you want to read. Even years only. 
        verbose -- (bool) set to True if you want it to print every file as it reads it in
    output:
        district_df -- pandas data frame with index in format ST_00_YYYY (e.g. AL_01_2018) with 
                       shapefiles stored in a column named 'shape'.
    """
    # read in the standard dictionary 
    district_df = pickle.load(open('../Datasets/master_index.p','rb'))
    district_df['shape'] = [np.nan]*district_df.shape[0] # make a blank column
    district_df['shape'] = district_df['shape'].astype(object) # reassign to object so it can hold shapely stuff

    for election_year in election_years:
        # convert election year to "Nth Congress" 
        congress_ID = congress_ID_list[election_year_list==election_year][0]

        # pre-2016 files come from http://cdmaps.polisci.ucla.edu/
        # they are a bit cleaner
        if election_year < 2016:
            # read in the shapefile (must be named 'districtsN.shp' in a folder titled 'districtShapesN')
            shpfilename = 'districtShapes-{0}/districts{0}.shp'.format(congress_ID)
            reader = shpreader.Reader(shpfilename) 
            districts = reader.records() # get full records
            geometries = reader.geometries() # get just the shape
            
            # put the shapefiles into the standard dictionary
            for record in reader.records(): # loop over districts
                attr = record.attributes # dictionary of information about the district
                poly = record.geometry # coordinates of the district as a shapely polygon

                # 1) get the state abbr of the district
                if any(state_names==attr['STATENAME'].upper()): # filter out districts that aren't in states
                    ST = state_abbrs[state_names==attr['STATENAME'].upper()][0]
                else: # pretty much just Washington, DC
                    print('{} is not a state.'.format(attr['STATENAME'].upper()))
                    continue
                # 2) get the id of the district
                id_int = int(attr['DISTRICT'])
                if id_int == 0: # change at-large district ID from 0 to 1 to play nice with our indexing convention.
                    id_int = 1 # todo: do we want to change this back? Looks like 0 for at-large districts is the convention. 
                ID = '{0:02d}'.format(id_int)
                # 3) put it all together into an index
                ind = '{}_{}_{}'.format(ST, ID, election_year)
                if verbose:
                    print('{} was read in.'.format(ind))
                # put the polygon in the dictionary
                district_df.at[ind,'shape'] = poly
            
        # 2016 and later are US Census Bureau Tiger Line Files 
        # (and they're a bit messier)
        if election_year >= 2016:
            # read in the shapefile (must be named 'tl_YYYY_us_cdN.shp' in a folder titled 't1_YYYY_us_cdN')
            shpfilename = 'tl_{1}_us_cd{0}/tl_{1}_us_cd{0}.shp'.format(congress_ID,election_year)
            reader = shpreader.Reader(shpfilename) 
            districts = reader.records() # get full records
            geometries = reader.geometries() # get just the shape

            # put the shapefiles into the standard dictionary
            for record in reader.records(): # loop over districts
                attr = record.attributes # dictionary of information about the district
                poly = record.geometry # coordinates of the district as a shapely polygon

                # 1) get the state abbr of the district
                if any(state_fips==int(attr['STATEFP'])): # filter out districts that aren't in states
                    ST = state_abbrs[state_fips==int(attr['STATEFP'])][0]
                else: # pretty much just Washington, DC
                    print('{} is not a state.'.format(int(attr['STATEFP'])))
                    continue
                # 2) get the id of the district
                if (attr['CD{}FP'.format(congress_ID)]=='ZZ'):
                    print('District ID is ZZ, which indicates this is not a congressional district.')
                    continue
                else:
                    id_int = int(attr['CD{}FP'.format(congress_ID)])
                if id_int == 0: # change at-large district ID from 0 to 1 to play nice with our indexing convention.
                    # todo: do we want to change this back? Looks like 0 for at-large districts is the convention. 
                    id_int = 1
                ID = '{0:02d}'.format(id_int)
                # 3) put it all together into an index
                ind = '{}_{}_{}'.format(ST, ID, election_year)
                if verbose:
                    print('{} was read in.'.format(ind))
                # put the polygon in the dictionary
                district_df.at[ind,'shape'] = poly
    
    return district_df

In [5]:
# find if a district has changed between last year and this year
def check_if_districts_changed(this_year, district_df, threshold_for_change=0.1):
    """
    Checks if each district changed more than the set threshold since the last election year.
    Change is a fraction of the district which overlaps with the previous year of that district (ranges 0-1).
    If the total area of the district increases, the overlap area is divided by the largest of the two years.
        (this prevents counting a district as unchanged if its area has increased)
    
    input:
        this_year -- (int) Elections years of the districts you want to check. Even years only. 
        district_df -- (pd dataframe) dataframe with default indices and shapefiles stored in 
                       a column named 'shape'
    output:
        district_df -- pandas data frame with index in format ST_00_YYYY (e.g. AL_01_2018) 
                       with change from last year stored in a column called 'border_change'.
                       Statuses:
                           'same' - this district has not changed at all since the previous year
                           'new' - indicates a district with this ID was not in this state last year
                           'changed' - indicates the borders have changed from last year, but that 
                                       this district was present in its state last year
    """
    # loop over states so you only have to compare districts in-state
    # otherwise, comparing each district to 434 other districts would be super slow
    for ST in state_abbrs: 
        prev_year = this_year-2
        # get the relevant districts
        districts = district_df.loc[np.logical_and(district_df['state']==ST,
                                                 district_df['year']==this_year)]
        districts_prev = district_df.loc[np.logical_and(district_df['state']==ST,
                                                 district_df['year']==prev_year)]
        # loop over districts in your current year
        for ind,district in districts.iterrows():
            # find previous year's district
            district_prev = districts_prev.loc[districts_prev['district']==district['district']]
            ind_prev = district_prev.index

            # determine whether district is new or borders have changed
            if ind_prev.shape[0]==0: # if the district didn't exist last year
                # then this district is new this year
                district_df.loc[ind,'border_change'] = 'new'
            else: 
                # check if the borders have changed
                shape_prev = district_prev['shape'].values[0]
                shape = district['shape']

                # check if shapes intersect with themselves
                if not (shape.is_valid and shape_prev.is_valid): 
                    # if they do, use buffer to correct this
                    print('The following polygons intersected with themselves. Attempting to buffer shape...')
                    if not shape.is_valid:
                        print(ind)
                        shape = shape.buffer(0)
                    if not shape_prev.is_valid:
                        print(ind_prev)
                        shape_prev = shape_prev.buffer(0)

                # calculate overlap percent
                area = shape.area # area of this district
                area_prev = shape_prev.area
                overlap_area = shape.intersection(shape_prev).area # area of overlap between shape and shape_prev
                frac_overlap = overlap_area/np.max([area,area_prev]) # fractional overlap between new and old district
                # divide by the larger of the old or new district
                
                if (1.-frac_overlap) < threshold_for_change: 
                    # then district has not changed
                    district_df.loc[ind,'border_change'] = 'same'
                else:
                    # district has changed
                    district_df.loc[ind,'border_change'] = 'changed'
                    print(ind)
                    print(frac_overlap)
                    
    return district_df

In [6]:
# compute overlap percent between this district and last year's districts
def district_overlap(this_year, district_df):
    """
    Finds the fractional overlap between this year's district and the previous year's districts. 
    i.e., if a Florida's 1st district has changed its borders from 2014 to 2016, it may be made up of:
        20% of its area may come from district 2 in 2014
        40% of its area may come from district 3 in 2014
        10% of its area may come from district 4 in 2014
    
    input:
        this_year -- (int) Elections years of the districts you want to check. Even years only. 
        district_df -- (pd dataframe) dataframe with default indices and shapefiles stored in 
                       a column named 'shape'
    output:
        district_df -- pandas data frame with index in format ST_00_YYYY (e.g. AL_01_2018) 
                       with fractional overlap stored in a column called 'fractional_overlap'.
                       
                       Each district has a dictionary where the keys are the indicies of the previous
                       districts which overlap with our district, and the values are the fractional overlap.
    """
    # loop over states so you only have to compare districts in-state
    # otherwise, comparing each district to 434 other districts would be super slow
    for ST in state_abbrs: 
        prev_year = this_year-2
        # get the relevant districts
        districts = district_df.loc[np.logical_and(district_df['state']==ST,
                                                 district_df['year']==this_year)]
        districts_prev = district_df.loc[np.logical_and(district_df['state']==ST,
                                                 district_df['year']==prev_year)]

        for ind,district in districts.iterrows(): # loop over districts in your current year
            overlap_dict = {}
            shape = district['shape']
            area = shape.area # area of this district
            # check if shapes intersect with themselves
            if not (shape.is_valid and shape_prev.is_valid): 
                # if they do, use buffer to correct this
                print('The following polygons intersected with themselves. Attempting to buffer shape...')
                if not shape.is_valid:
                    print(ind)
                    shape = shape.buffer(0)
                if not shape_prev.is_valid:
                    print(ind_prev)
                    shape_prev = shape_prev.buffer(0)
            for ind_prev,district_prev in districts_prev.iterrows(): # loop over districts in previous year
                shape_prev = district_prev['shape']

                # check if shapes intersect with themselves
                if not (shape.is_valid and shape_prev.is_valid): 
                    # if they do, use buffer to correct this
                    print('The following polygons intersected with themselves. Attempting to buffer shape...')
                    if not shape.is_valid:
                        print(ind)
                        shape = shape.buffer(0)
                    if not shape_prev.is_valid:
                        print(ind_prev)
                        shape_prev = shape_prev.buffer(0)

                # calculate frac overlap
                area_prev = shape_prev.area
                overlap_area = shape.intersection(shape_prev).area # area of overlap between shape and shape_prev
                frac_overlap = overlap_area/area # fractional overlap between new and old district

                if frac_overlap > 0.:
                    overlap_dict[ind_prev] = frac_overlap

            print(ind)
            print(overlap_dict)
            district_df.at[ind, 'overlap_frac'] = overlap_dict
            
    return district_df

In [51]:
def get_centroid(district_df):
    """
    Finds the centroid of a district in lon,lat. 
    
    input: 
        district_df -- (pd dataframe) dataframe with default indices and shapefiles stored in 
                       a column named 'shape'
    output:
        district_df -- pandas data frame with index in format ST_00_YYYY (e.g. AL_01_2018) 
                       with centroid stored in a column called 'centroid'.
                       The centroid is calculated as a lon,lat on a Cartesian plane.
                       It ignores spherical geometry. 
                       The centroid is stored as a tuple in the form (lon,lat)
    """
    # add column
    district_df['centroid'] = [np.nan]*district_df.shape[0] # make a blank column
    district_df['centroid'] = district_df['centroid'].astype(object) # reassign to object so it can hold shapely stuff
    
    for ind, district in district_df.iterrows():
        shape = district['shape']
        if pd.isnull(district['shape']): # if there's no shape, fill it with a nan
            district_df.at[ind, 'centroid'] = np.nan
        else:
            centroid = shape.centroid.coords
            district_df.at[ind, 'centroid'] = centroid # in units of lat/lon
            
    return district_df

In [ ]:
def get_distance(district_df):
    # add column
    #todo

In [54]:
# get centroid coords for everything 
years = [2002, 2004, 2006, 2008, 2010, 2012, 2014, 2016, 2018]

centroid_df = read_shapefiles(years, verbose=False)
for year in years:
     centroid_df = get_centroid(centroid_df)
pickle.dump(centroid_df, open('centroid.p', 'wb'))

DISTRICT OF COLUMBIA is not a state.
DISTRICT OF COLUMBIA is not a state.
DISTRICT OF COLUMBIA is not a state.
DISTRICT OF COLUMBIA is not a state.
DISTRICT OF COLUMBIA is not a state.
DISTRICT OF COLUMBIA is not a state.
DISTRICT OF COLUMBIA is not a state.
11 is not a state.
60 is not a state.
66 is not a state.
69 is not a state.
72 is not a state.
78 is not a state.
District ID is ZZ, which indicates this is not a congressional district.
District ID is ZZ, which indicates this is not a congressional district.
District ID is ZZ, which indicates this is not a congressional district.


ValueError: Must have equal len keys and value when setting with an iterable

In [ ]:
# find out if each district changed
years = [2002, 2004, 2006, 2008, 2010, 2012, 2014, 2016, 2018]

# read in the data (make a fresh df)
change_df = read_shapefiles(years, verbose=False)
for year in years:
    change_df = check_if_districts_changed(this_year, change_df)
pickle.dump(change_df, open('change.p', 'wb'))

In [ ]:
# find out the area overlap between this year's and last year's districts
years = [2002, 2004, 2006, 2008, 2010, 2012, 2014, 2016, 2018]

# read in the data (make a fresh df)
overlap_df = read_shapefiles(years, verbose=False)
for year in years:
    overlap_df = district_overlap(this_year, overlap_df, threshold_for_change=0.1)
pickle.dump(overlap_df, open('overlap_frac.p', 'wb'))

In [ ]:
# if all of these miraculously run without errors, pickle them in a combined df
all_overlap_data_df = change_df.copy()
all_overlap_data_df['overlap_frac'] = overlap_df['overlap_frac']
all_overlap_data_df['centroid'] = overlap_df['centroid']
pickle.dump(all_overlap_data_df, open('all_overlap_data.p', 'wb'))